<a href="https://colab.research.google.com/github/kvsnoufal/MSPhi2-for-classification-LoRA/blob/main/trainPhi2Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import json
json_file_path = "/home/sy/projects/docAI/dataset/document_summary/Training/article_train_original.json"
save_path = "/home/sy/projects/docAI/dataset/document_summary/preprocessing"
with open(json_file_path,'r') as json_file:
    json_dict = json.load(json_file)
    documents = json_dict['documents']


with open(f"{save_path}/article_train_docs.json",'w') as json_file:
    json.dump(documents, json_file)

In [21]:
import os
json_dir = "/home/sy/projects/docAI/dataset/document_summary/Validation"
file_list = os.listdir(json_dir)
documents = None
for file_name in file_list:
    if file_name.find('.json') >= 0:
        json_file_path = os.path.join(json_dir, file_name)
        with open(json_file_path,'r', encoding='utf-8') as json_file:
            json_dict = json.load(json_file)
            if documents == None:
                documents = json_dict['documents']
            else:
                documents = documents + json_dict['documents']

with open(f"{save_path}/merge_valid_docs.json",'w', encoding='utf-8') as json_file:
    json.dump(documents, json_file)

In [10]:
with open("/home/sy/projects/docAI/dataset/document_summary/preprocessing/merge_train_docs.json",'r') as json_file:
    json_dict = json.load(json_file)
    documents1 = json_dict

with open("/home/sy/projects/docAI/dataset/document_summary/preprocessing/merge_valid_docs.json",'r') as json_file:
    json_dict = json.load(json_file)
    documents2 = json_dict
merge_doc = documents1 + documents2
with open(f"{save_path}/merge_all_docs.json",'w') as json_file:
    json.dump(merge_doc, json_file)

In [26]:
# data loading
from datasets import load_dataset
dataset = load_dataset("json", data_files="/home/sy/projects/docAI/dataset/document_summary/preprocessing/article_valid_docs.json")
# Split the dataset into training and validation datasets
data = dataset['train'].train_test_split(train_size=0.8, seed=42)
data['val'] = data['test']
# explore data
data.items()
print(data['train'].to_pandas().info())
print(data['val'].to_pandas().info())
print(data['test'].to_pandas().info())
print(data['train'].to_pandas()['title'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24097 entries, 0 to 24096
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   title                    24097 non-null  object
 1   annotator_id             24097 non-null  int64 
 2   media_sub_type           24097 non-null  object
 3   media_name               24097 non-null  object
 4   category                 24097 non-null  object
 5   size                     24097 non-null  object
 6   document_quality_scores  24097 non-null  object
 7   id                       24097 non-null  object
 8   publish_date             24097 non-null  object
 9   abstractive              24097 non-null  object
 10  char_count               24097 non-null  object
 11  extractive               24097 non-null  object
 12  text                     24097 non-null  object
 13  media_type               24097 non-null  object
dtypes: int64(1), object(13)
memory usage: 

In [28]:
data['train'].to_pandas()['text']

0        [[{'highlight_indices': '33,35', 'index': 0, '...
1        [[{'highlight_indices': '', 'index': 0, 'sente...
2        [[{'highlight_indices': '36,38;53,55;60,63', '...
3        [[{'highlight_indices': '', 'index': 0, 'sente...
4        [[{'highlight_indices': '35,36', 'index': 0, '...
                               ...                        
24092    [[{'highlight_indices': '', 'index': 0, 'sente...
24093    [[{'highlight_indices': '', 'index': 0, 'sente...
24094    [[{'highlight_indices': '', 'index': 0, 'sente...
24095    [[{'highlight_indices': '', 'index': 0, 'sente...
24096    [[{'highlight_indices': '', 'index': 0, 'sente...
Name: text, Length: 24097, dtype: object

### Data prep

In [27]:
# tokenizer
from transformers import AutoTokenizer
model_id = "beomi/KoAlpaca-Polyglot-5.8B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True,)
# set pad token - to avoid error while training
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [48]:
import random
MAX_LEN = 1024
def preprocessing_function(examples):
    texts = examples['text']

    descriptions = ""

    for text in texts:
        if len(descriptions) != 0:
            descriptions = descriptions + '\n'
        for text_line in text:
            if len(descriptions) == 0:
                descriptions = text_line['sentence']
            else:
                descriptions = descriptions + ' ' + text_line['sentence']
        
        

    prompt = f"제목 : {examples['title']}\n본문: {descriptions}\n요약: {examples['abstractive']}"
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=MAX_LEN)


    return tokenized
# test
tokenized = preprocessing_function(data['train'][4])
len(tokenized['input_ids'])

1024

In [50]:
decoded_sequence = tokenizer.decode(tokenized['input_ids'])
print(decoded_sequence)

제목 : 윤지오 상대 손배 소송 ‘봇물’... 혈세 반환엔 경찰 “방법 없다“
본문: [헤럴드경제=박병국 기자] 윤지오씨에 대한 '후원금 반환 소송'이 줄을 잇고 있지만, 윤씨 보호에 사용된 혈세는 돌려받을 길이 없는 것으로 확인됐다. 경찰은 윤씨 보호에 모두 900만원 가량을 지출했다. 경찰은 또 국회로부터 윤씨 보호에 사용된 지원금 반환 가능성을 묻는 질의가 들어왔지만 이에 대해 '반환받기는 어렵다'는 입장을 국회에 제출한 것으로 알려졌다.
 경찰청 관계자는 10일 헤럴드경제와의 통화에서 "윤 씨가 국가를 속여서 신변보호를 받았다고 하면 국가가 윤 씨에 대해 민사상 문제를 제기 할 수 있지만 경찰이 심사를 했고 특정한 정도의 수준으로 신변보호를 해야 한다고 저희가 판단을 했기 때문에 현재로서는 지원금을 돌려받을 방법이 없다"고 했다.
 이 관계자는 또 윤 씨의 후원자들이 후원금 반환 소송을 제기한 것과 관련해서는 "후원금 반환 소송과는 다른 문제다. 경찰이 윤 씨를 지원한 것은 신변 안전과 관련된 부분"이라고 했다. 그는 "윤 씨에 대한 지원금을 돌려받을 수 있는지에 대한 법리적 검토를 했지만 방법이 없다"며 "국회에서 윤 씨 지원에 쓰인 예산을 반환할 방법이 없냐는 질의가 이어지고 있지만 방법이 없다는 내용을 전달하고 있다"고 했다.
 법률사무소 로앤어스는 윤씨에게 후원금을 반환하라는 손해배상 청구소송을 10일 서울중앙지법에 내기로 했다. 전날까지 소송에 참여의사를 밝힌 인원만 500명이다. 이번 소송에 참여한 후원자 중 대부분이 1000원~1만원 단위 소액을 후원한 것으로 알려졌다. 후원자들은 윤씨의 후원금 모금 목적이 불순했고 사용 내역도 공개하고 있지 않고 있어 의심스럽다며 소 제기 이유를 설명했다. 지난 4월에는 윤 씨의 출판을 도운 김수민 작가가 "윤지오는 고 장자연의 억울한 죽음을 이용하고 있다"며 윤 씨를 고소하기도 했다.
 경찰은 지난 3월 15일부터 4월 23일까지 40일간 윤 씨가 머문 호텔비 등을 지원했다. 이 과정에서 총 927만40

In [51]:
import os

col_to_delete = data['train'].to_pandas().columns.to_list()
# Apply the preprocessing function and remove the undesired columns
tokenized_datasets = data.map(
    preprocessing_function, 
    batched=False, 
    remove_columns=col_to_delete, 
    num_proc=int(os.cpu_count()/2))
# Rename the target to label as for HugginFace standards
# tokenized_datasets = tokenized_datasets.rename_column("target", "label")
# Set to torch format
tokenized_datasets.set_format("torch")
# test
tokenized_datasets['train'][0]

Map (num_proc=12):   0%|          | 0/24097 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/6025 [00:00<?, ? examples/s]

{'input_ids': tensor([ 400,  919, 4361,  ...,    2,    2,    2]),
 'token_type_ids': tensor([0, 0, 0,  ..., 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0])}

In [42]:
tokenized_datasets['train'].to_pandas()

,input_ids,token_type_ids,attention_mask
0,"[400, 919, 4361, 6016, 1411, 15, 2306, 21177, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[400, 919, 4361, 2410, 2474, 15, 4410, 1216, 5...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[400, 919, 4361, 24769, 457, 5808, 1411, 6487,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[400, 919, 4361, 5915, 2314, 13260, 8243, 642,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[400, 919, 4361, 1557, 9470, 1640, 966, 942, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...
24092,"[400, 919, 4361, 514, 2855, 525, 1758, 13782, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
24093,"[400, 919, 4361, 975, 9766, 15, 4491, 12445, 6...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
24094,"[400, 919, 4361, 3327, 1313, 8611, 1878, 13905...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
24095,"[400, 919, 4361, 471, 1644, 417, 1351, 465, 43...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [43]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/KoAlpaca-Polyglot-5.8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

In [44]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [45]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [46]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3670016 || all params: 3070156800 || trainable%: 0.11953838970048696


In [52]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_datasets['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        max_steps=5150,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        logging_dir="./doc_summary/logs",
        output_dir="./doc_summary",
        # optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

max_steps is given, it will override any value given in num_train_epochs
/root/anaconda3/envs/laylm_cls/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/anaconda3/envs/laylm_cls/lib/python3.8/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,3.752700
20,3.500700
30,3.135700
40,2.815800
50,2.666700
60,2.583000
70,2.455500
80,2.381000
90,2.426300
100,2.398600


/root/anaconda3/envs/laylm_cls/lib/python3.8/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/anaconda3/envs/laylm_cls/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/anaconda3/envs/laylm_cls/lib/python3.8/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exc

TrainOutput(global_step=5150, training_loss=2.229791190878859, metrics={'train_runtime': 18502.7396, 'train_samples_per_second': 2.227, 'train_steps_per_second': 0.278, 'total_flos': 1.4591946139222671e+18, 'train_loss': 2.229791190878859, 'epoch': 1.7092598738798541})

In [53]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [2]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            x,
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=1024,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    gened = tokenizer.decode(gened[0])
    split_gened = gened.split('\n')
    summary = split_gened[-1]

    summary_list = summary.split('요약: ')
    output_text = ""
    for idx, summary in  enumerate(summary_list):
        output_text += f"{idx+1}. {summary}\n"

    print(gened)

def preprocessing_function_test(examples):
    texts = examples['text']

    descriptions = ""

    for text in texts:
        if len(descriptions) != 0:
            descriptions = descriptions + '\n'
        for text_line in text:
            if len(descriptions) == 0:
                descriptions = text_line['sentence']
            else:
                descriptions = descriptions + ' ' + text_line['sentence']
        
        

    prompt = f"제목 : {examples['title']}\n본문: {descriptions}\n요약:"
    tokenized = tokenizer(prompt, truncation=True, max_length=MAX_LEN)


    return tokenized

In [90]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

def art_crawl(url):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수 
    
    Args: 
        all_hrefs(dict): 각 분야별로 100페이지까지 링크를 수집한 딕셔너리 (key: 분야(sid), value: 링크)
        sid(int): 분야 [100: 정치, 101: 경제, 102: 사회, 103: 생활/문화, 104: 세계, 105: IT/과학]
        index(int): 링크의 인덱스
    
    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리
    
    """
    art_dic = {}
    
    ## 1.
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"
    
    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    
    ## 2.
    # 제목 수집
    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)
    
    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)
    
    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)
    
    ## 3.
    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str
    
    return art_dic

art_crawl("https://n.news.naver.com/article/022/0003945184?cds=news_media_pc&type=editn")

{'title': '‘손흥민 아버지’ 손웅정, 아동학대 혐의 피소…“고소인 주장과 달라” 반박',
 'date': '2024.06.26. 오전 6:45',
 'main': '손 감독 “사랑 전제된 언행…시대 변화 못 읽은 점 반성”손흥민의 아버지 손웅정 감독이 운영하는 유소년 축구 훈련기관 \'SON축구아카데미\'에서 손 감독과 코치진들이 소속 유소년 선수에 대한 욕설과 체벌 등 아동학대 혐의로 피소됐다.  \xa0  손 감독은 "최근 아카데미 훈련 도중 거친 표현과 체력 훈련 중 이뤄진 체벌에 관해 현재 수사가 진행 중"이라며 "마음의 상처를 받은 아이와 그 가족들에게 깊은 사과의 뜻을 전한다"고 밝혔다.  \n\n\n\n   손웅정 SON축구아카데미 감독. 연합뉴스.    26일 연합뉴스에 따르면 손 감독과 A 코치, B 코치 등 3명은 아동복지법상 아동학대 혐의로 송치돼 검찰 조사를 받는 것으로 확인됐다.  \xa0  이 사건은 지난 3월 19일 아동 C군 측이 "오키나와 전지훈련 중이던 지난 3월 9일 A 코치가 C군의 허벅지 부위를 코너킥 봉으로 때려 2주간 치료가 필요한 상처를 입혔다"고 고소하면서 불거졌다.  \xa0  고소인 측이 경찰 조사에서 진술한 바에 따르면 당시 경기에서 진 C군 팀 선수들은 패배했다는 이유로 A 코치로부터 정해진 시간 내에 골대에서 중앙선까지 20초 안에 뛰어오라는 지시를 받았다고 주장했다.  \xa0  그러나 C군을 비롯한 4명이 제시간에 들어오지 못하자 엎드린 자세로 엉덩이를 코너킥 봉으로 맞았다고 진술했다.  \xa0  손 감독으로부터도 오키나와 전지훈련 기간이었던 지난 3월 7∼12일 훈련 중 실수했다는 이유로 욕설을 들은 것을 비롯해 경기는 물론 기본기 훈련을 잘 못한다는 이유로 욕을 들었다는 내용이 진술에 포함됐다.  \xa0  아카데미 소속 선수들이 함께 사는 숙소에서 B 코치에 의해 엉덩이와 종아리를 여러 차례 맞았고, 구레나룻을 잡아당기거나 머리 부위를 맞았다는 주장도 진술서에 담겼다.  \n\n\n\n 

In [4]:
artdic = art_crawl("https://n.news.naver.com/article/022/0003945184?cds=news_media_pc&type=editn")
artdic

NameError: name 'art_crawl' is not defined

In [3]:
# tokenized = preprocessing_function_test(data['test'][4])
# decoded_sequence = tokenizer.decode(tokenized['input_ids'])
# print(decoded_sequence)

# title = '“열받아서 욕했어요” 퍼펙트 깨지고 1피안타 완봉승, 65억 포수 왜 “엄청 즐겁지 않네요” 풀죽었을까'
# description = '[OSEN=잠실, 한용섭 기자] 8회까지 24타자 연속 범타로 처리하고, 9회 선두타자에게 안타를 맞아 퍼펙트가 무산됐다. 퍼펙트가 깨지는 순간, 마운드의 투수 만큼이나 포수도 진한 아쉬움을 느꼈다. 프로야구 LG 트윈스 박동원은 25일 잠실구장에서 열린 삼성과 경기에서 4-0으로 승리한 후에 마치 패배한 듯 한 표정이었다.  선발 투수 켈리는 이날 9회 선두타자 윤정빈에게 체인지업 2개를 연속으로 던졌다가 중전 안타를 맞고 첫 출루를 허용, 대망의 퍼펙트가 깨졌다. 43년 만에 KBO리그 최초 퍼펙트 달성을 눈앞에 뒀으나 아쉽게 실패했다.  켈리는 마운드에서 허리를 숙이며 한 마디 내뱉었다. 경기 후 켈리는 안타를 허용하고 했던 말이 무엇인지, 취재진 인터뷰 자리에 함께 한 두 자녀 앞에서 할 수 있는 말인지 질문을 받자 “말하기 어렵다. 다들 무엇인지 알거다”라고 웃으며 말했다. 비속어 ‘FXXK’을 내뱉은 것. 박동원도 마찬가지였다. 퍼펙트가 깨지자 박동원은 마운드로 올라갔고, 김경태 투수코치가 덕아웃에서 나와 분위기를 진정시켰다. 박동원은 켈리와 무슨 이야기를 나눴는지 묻자 “완봉하자고 했다. 욕도 좀 하고, 열받아서 아쉬워서 그랬다(욕을 했다)”며 “한 번에 다 무너진 것 같다. 예전에 8회 2사까지 퍼펙트 한 적이 있었다”고 허탈하게 말했다. 박동원이 넥센에서 뛸 때 2018년 4월 18일 NC와 경기에서 투수 최원태와 8회 1사까지 퍼펙트, 최준석에게 안타를 맞고 무산됐다. 경기 도중 9회 이전에 켈리와 얘기를 나눈 것은 딱 한 번이었다고. 박동원은 “한 마디 정도 했다. 초반에 좀 아쉬운 게 있어서 3회였나, 맥키넌 타석 지나고 한 번 얘기한 뒤로는 한 마디도 안 했다”고 말했다.7회 정도부터 퍼펙트를 의식했다고 한다. 박동원은 “퍼펙트는 안 되더라도 노히트 노런은 했어야 하는데 너무 아쉽네요. 열심히 했는데. 이기고 완봉해서 너무 좋은데, 엄청 즐겁지가 않네요. 꿈에 그리던 퍼펙트인데, 꿈이 깨져버렸으니까”라고 힘없이 말했다.  켈리의 구위와 컨디션이 너무 좋았다. 직구 최고 구속이 149km까지 나왔고 직구 37개, 커브 25개, 체인지업 19개, 슬라이더 16개, 포크볼 3개, 투심 2개를 던졌다. 박동원은 “오늘 켈리 공은 커브가 너무 좋았다. 체인지업도 좋았고, 포크볼도 좀 던지고 다 좋았다. 직구도 되게 좋았고, 모든 것이 완벽했으니까 좋은 결과를 내지 않았을까요”고 말했다.'
remove_enter = ['\n','\n\n','\n\n\n','\n\n\n\n','\n\n\n\n\n','\n\n\n\n\n\n']

for rm_ent in remove_enter:
    description = artdic['main'].replace(rm_ent, '\n')

prompt = f"제목 : {artdic['title']}\n본문: {description}\n요약:"

# print(prompt)
gen(f"요약해줘 \n{prompt}")

NameError: name 'artdic' is not defined